In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1002"

convolution length = 3

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [7]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[3, n_variables, 32],
                 [3, 32, 64],
                 [3, 64, 64],
                 [3, 64, 64],
                 [3, 64, 64]]
bias_shape = [32, 64, 64, 64, 64]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [8]:
print pos_weight

3.18601822124


In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [11]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [12]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [13]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [14]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.281286, fmeasure_max= 0.459519, precision= 0.325480, recall= 0.781250
Train Random guess:  , auc= 0.267409fmeasure_max= 0.416667, precision= 0.263365, recall= 0.997024
Validation Iter 0, auc= 0.228602, fmeasure_max= 0.378361, precision= 0.233887, recall= 0.989708
Validation Random guess:  auc= 0.234443, fmeasure_max= 0.371574, precision= 0.228180, recall= 1.000000
Minibatch Loss= 1.095550
Tarin Iter 100, auc= 0.456565, fmeasure_max= 0.467946, precision= 0.329598, recall= 0.806452
Train Random guess:  , auc= 0.243083fmeasure_max= 0.391416, precision= 0.244205, recall= 0.985484
Validation Iter 100, auc= 0.370511, fmeasure_max= 0.418196, precision= 0.272491, recall= 0.898799
Validation Random guess:  auc= 0.231267, fmeasure_max= 0.371840, precision= 0.228560, recall= 0.996569
Minibatch Loss= 1.009241
Tarin Iter 200, auc= 0.544068, fmeasure_max= 0.560124, precision= 0.498619, recall= 0.638938
Train Random guess:  , auc= 0.221618fmeasure_max= 0.362003, precision= 0.2210

In [15]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

ValueError: Parent directory of ../model/CNN1002/CNN1002 doesn't exist, can't save.

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [16]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.604874 F1_max = 0.564402 prec = 0.496852 rec = 0.653209
Test Random guess: auc= 0.236752 fmeasure_max= 0.375691 , precision= 0.231329 , recall= 0.999324


In [17]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [18]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

IOError: Unable to create or open file.